In [3]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import numpy as np


In [4]:
imported_to='2023-03-13'
# Get Last Upldate from BQ update data
dateCols=['open_datetime','close_datetime','response_datetime','resolved_datetime']

removeCols=dateCols+['open_to_close','response_to_resolved']

numbericCols=['open_to_close_hour','response_to_resolved_hour']
cateCols=['sla','product_type','brand','service_type','incident_type']

file_name="New_Incident.csv"
table_id = "pongthorn.SMartML.new_incident"

client = bigquery.Client()


In [5]:
sql=f"""
SELECT  id,
severity_id,severity_name,sla,
product_type,brand,service_type,incident_type,
open_datetime,  close_datetime, response_datetime,resolved_datetime, imported_at
FROM `pongthorn.SMartDW.incident` WHERE imported_at>= '{imported_to}'
order by imported_at
"""


query_result=client.query(sql)
df_all=query_result.to_dataframe()
df_all.head()

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_datetime,close_datetime,response_datetime,resolved_datetime,imported_at
0,2423,4,Cosmatic,24x7 4Hrs Response Time,Software,VMWare,Request,Configuration Change,2023-03-10 23:00:00,2023-03-11 10:30:00,2023-03-10 23:00:00,2023-03-11 10:30:00,2023-03-13 21:00:05.778139
1,2429,4,Cosmatic,24x7 4Hrs Resolution Time,Storage,HPE,Request,Upgrade Software,2023-03-10 22:00:00,2023-03-11 03:00:00,2023-03-10 22:01:00,2023-03-11 03:00:00,2023-03-13 21:00:05.778139
2,2421,2,Major,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,2023-03-10 16:23:00,2023-03-13 22:45:00,2023-03-10 16:33:00,2023-03-10 16:45:00,2023-03-13 21:00:05.778139
3,2342,2,Major,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,2023-02-19 21:24:00,2023-03-13 17:35:00,2023-02-19 21:32:00,2023-03-13 17:35:00,2023-03-13 21:00:05.778139
4,1867,4,Cosmatic,24x7 4Hrs Resolution Time,Other,F5,Request,Configuration Change,2022-11-18 15:17:00,2023-03-13 12:15:00,2022-11-18 15:21:00,2023-01-23 13:48:00,2023-03-13 21:00:05.778139


In [6]:
start_end_list=[ ['open_datetime','close_datetime'],['response_datetime','resolved_datetime']]
listDiffDateDeltaCols=[]
listDiffHourCols=[]
for item  in  start_end_list:
   diff_str=f"{item[0]}_to_{item[1]}" 
   diff_str=diff_str.replace('_datetime','')  
   listDiffDateDeltaCols.append(diff_str)
   df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
   diff_hour=f'{diff_str}_hour'
   listDiffHourCols.append(diff_hour)
   df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )


In [7]:
print(f"Max Last Import ={df_all['imported_at'].max()}")

for col in numbericCols:
 df_all=df_all.query(f'{col}!=0')

df_all=df_all.drop_duplicates(subset=['id'],keep='first')
df_all=df_all.drop(columns=removeCols)

print(df_all.info())
df_all.tail()

Max Last Import =2023-03-22 21:00:07.048396
<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 84
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         81 non-null     Int64         
 1   severity_id                81 non-null     Int64         
 2   severity_name              81 non-null     object        
 3   sla                        81 non-null     object        
 4   product_type               81 non-null     object        
 5   brand                      81 non-null     object        
 6   service_type               81 non-null     object        
 7   incident_type              81 non-null     object        
 8   imported_at                81 non-null     datetime64[ns]
 9   open_to_close_hour         81 non-null     float64       
 10  response_to_resolved_hour  81 non-null     float64       
dtypes: Int64(2), datetime64[ns](1

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,imported_at,open_to_close_hour,response_to_resolved_hour
80,2452,2,Major,24x7 4Hrs Response Time,Server,HPE,Incident,Software,2023-03-21 18:29:01.995333,4.500000,4.000000
81,2491,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Request,Hard Disk Drive Failure,2023-03-21 18:29:01.995333,7.666667,7.666667
82,2508,3,Minor,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,2023-03-22 21:00:07.048396,1.750000,1.733333
83,2509,3,Minor,24x7 6Hrs Resolution Time,Software,CIMCO-CMMS,Incident,Software,2023-03-22 21:00:07.048396,750.250000,73.283333
84,2498,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,Hard Disk Drive Failure,2023-03-22 21:00:07.048396,13.666667,13.666667


In [11]:
df_all[numbericCols].describe(percentiles=[.9,.75,.50,.25,.10])

,open_to_close_hour,response_to_resolved_hour
count,81.000000,81.000000
mean,301.525926,271.728395
std,1499.015346,1480.571763
min,0.083333,0.066667
10%,0.233333,0.200000
25%,4.100000,1.733333
50%,12.150000,8.983333
75%,80.033333,69.766667
90%,524.183333,313.000000
max,13177.250000,13176.250000


In [14]:
# df_all.to_csv("data/New_Incident.csv",index=False)
# dfError= df_all.query("open_to_close_hour>=524.183333")
# dfError.to_csv("Error_Incident.csv",index=False)
dfError
# dfError['id'].tolist()

In [15]:

def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df_all, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_all), "Imported igquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex


Total  81 Imported igquery successfully


,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,imported_at,open_to_close_hour,response_to_resolved_hour
3,2342,2,Major,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,2023-03-13 21:00:05.778139,524.183333,524.050000
4,1867,4,Cosmatic,24x7 4Hrs Resolution Time,Other,F5,Request,Configuration Change,2023-03-13 21:00:05.778139,2756.966667,1582.450000
15,2084,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,2023-03-14 21:00:06.414674,1485.650000,1485.650000
36,2279,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,868.283333,868.133333
38,2456,2,Major,24x7 6Hrs Resolution Time,Software,SAPB1,Request,Configuration Change,2023-03-20 21:00:07.377573,769.083333,749.683333
43,100,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,13177.250000,13176.250000
45,2229,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,874.033333,873.516667
67,2341,2,Major,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,2023-03-21 18:29:01.995333,627.783333,627.566667
83,2509,3,Minor,24x7 6Hrs Resolution Time,Software,CIMCO-CMMS,Incident,Software,2023-03-22 21:00:07.048396,750.250000,73.283333
